In [1]:
import os
import sys
import torch

In [2]:
IN_COLAB='google.colab' in sys.modules

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia024_1s_2024/trabalho_final/llm_editing_evaluation/rome/notebooks"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    API_KEYS_FILE="../../ia368v_dd/api_keys_20240427.json"

    TORCH_DTYPE=torch.float32

    sys.path.append("/content/drive/MyDrive/unicamp/ia024_1s_2024/trabalho_final/llm_editing_evaluation/rome/notebooks/rome")

    !pip install datasets -q
else:
    WORKING_FOLDER="/work/ia024_1s_2024/trabalho_final/llm_editing_evaluation/rome/notebooks"

    API_KEYS_FILE="../../api_keys_20240427.json"

    TORCH_DTYPE=torch.float16 # To Phi 1.5 fit 6GB GPU

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [4]:
import numpy as np
import json
import pandas as pd

from transformers import AutoModelForCausalLM, AutoTokenizer

import pickle
import time

from tqdm.auto import tqdm

import pandas as pd

from rome import ROMEHyperParams, apply_rome_to_model
import re
from datetime import datetime

In [5]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [6]:
os.getcwd()

'/content/drive/MyDrive/unicamp/ia024_1s_2024/trabalho_final/llm_editing_evaluation/rome/notebooks'

In [7]:
MODEL_NAME="microsoft/phi-1_5" # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B

MODEL_ROME_HYPERPARAMS="../hparams/ROME/microsoft_phi-1_5.json"

In [8]:
BASE_FACTUAL_ASSOCIATIONS="data/extracted_3_step_factual_associations_20240625.pkl"

## Load unchanged model

In [9]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=TORCH_DTYPE).to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [10]:
tok = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

### Configure a padding token for the tokenizer

In [11]:
tok.pad_token = tok.eos_token

## Load factual associations to add to the model

In [14]:
with open(BASE_FACTUAL_ASSOCIATIONS, "rb") as input_file:
    base = pickle.load(input_file)

In [15]:
base.keys()

dict_keys(['facts', 'fact_results', 'questions', 'questions_results', 'questions_from_facts', 'questions_from_facts_results'])

In [16]:
base['facts']

[{'subject': 'Abhay Bhushan Pandey',
  'relation': 'is',
  'object': 'an Indian computer scientist'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'made significant contributions to the development of the',
  'object': 'Internet TCP/IP architecture'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'is the author of the',
  'object': 'File Transfer Protocol and early versions of email protocols'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'graduated from the',
  'object': 'Indian Institute of Technology Kanpur in 1965 with a B.Tech in electrical engineering'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'received a Masters in electrical engineering and a degree in Management from the',
  'object': 'MIT Sloan School of Management'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'worked on developing FTP and email protocols for',
  'object': 'ARPANet and subsequent Internet'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'was a Director at the',
  'obj

## Support functions

### Ask a question to the model

In [17]:
def ask_question(which_model,
                 which_tok,
                 question,
                 max_new_tokens=20,
                 temperature=0.7,
                 verbose=True):

    model_inputs = which_tok(question, return_tensors='pt').to("cuda")

    sample_output = which_model.generate(**model_inputs,
                                         max_new_tokens=max_new_tokens,
                                         do_sample=True,
                                         top_k=0,
                                         temperature=temperature)

    decoded_answer = which_tok.decode(sample_output[0], skip_special_tokens=True)[len(question):]

    m = re.match("[\n|\r|\s]*(Answer:)?(A:)?\s*(.+)[\n]*", decoded_answer)

    if verbose:
        print("Decoded answer:\n{}\n".format(decoded_answer))

        print(m.groups())

        print(m.group(3))

    return m.group(3)

### Test the function

In [18]:
ask_question(model,
             tok,
             "What is Abhay Bhushan Pandey's profession?")

Decoded answer:

A: Abhay Bhushan Pandey is a professor of art history at the University

(None, 'A:', 'Abhay Bhushan Pandey is a professor of art history at the University')
Abhay Bhushan Pandey is a professor of art history at the University


'Abhay Bhushan Pandey is a professor of art history at the University'

### Function to format the ROME request

In [19]:
def create_edit_request(relation):
    return {'prompt': "{{}} {}".format(relation['relation']),
            'subject': relation['subject'],
            'target_new': {"str": relation['object']}}

### Ask list of questions

In [20]:
def ask_questions(which_model,
                  which_tok,
                  questions,
                  question_replicas=5):

    start_time = time.time()

    answers = []

    for question in questions:

        print("--------------")
        print(">> question: {}".format(question['question']))

        question_answers = []

        for i in range(question_replicas):
            question_answers.append(ask_question(which_model,
                                                 which_tok,
                                                 question['question']))

        answers.append({"question": question,
                        "answers": question_answers})

    end_time = time.time() - start_time

    print("Number of questions: {}; total time: {}".format(len(questions), end_time))

    return answers, end_time

### Ask statements questions

In [21]:
def ask_statements_questions(which_model,
                             which_tok,
                             questions,
                             question_replicas=5):

    start_time = time.time()

    model_answers = []

    for j, statement_questions in enumerate(questions):

        print("\n\n*******************************************")
        print("{} ― Questions for statement: {}".format(j, statement_questions['statement']))
        print("*******************************************\n")

        statement_answers, _ = ask_questions(which_model,
                                             which_tok,
                                             statement_questions['questions'],
                                             question_replicas=question_replicas)

        model_answers.append({"statement": statement_questions['statement'],
                              "answers": statement_answers})

    end_time = time.time()

    return model_answers, end_time - start_time

### Edit and Test model performance

In [22]:
def edit_and_test_model(which_model,
                        which_tok,
                        fact,
                        questions,
                        rome_hyperparams,
                        replicas=5):

    statement = "{} {} {}".format(fact['subject'],
                                  fact['relation'],
                                  fact['object'])

    edit_request = create_edit_request(fact)

    print(edit_request)

    edit_start_time = time.time()

    edit_result = apply_rome_to_model(model,
                                      tok,
                                      [edit_request],
                                      rome_hyperparams)

    edit_end_time = time.time()

    model_answers, questions_time = ask_statements_questions(which_model,
                                                             which_tok,
                                                             questions,
                                                             question_replicas=replicas)

    return {"statement": statement,
            "model_answers": model_answers,
            "edit_time": edit_end_time - edit_start_time,
            "questions_time": questions_time}

## Get textual model name and timestamp of the test round

In [23]:
test_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

textual_model_name = MODEL_NAME.replace("/", "_")

## Check model performance without editing for the factual associations questions

In [24]:
factual_original_answers, \
factual_questions_time = ask_statements_questions(model,
                                                  tok,
                                                  base['questions_from_facts'])



*******************************************
0 ― Questions for statement: Abhay Bhushan Pandey is an Indian computer scientist
*******************************************

--------------
>> question: Who is Abhay Bhushan Pandey?
Decoded answer:


Abhay Bhushan Pandey is a renowned scientist and entrepreneur who has made significant

(None, None, 'Abhay Bhushan Pandey is a renowned scientist and entrepreneur who has made significant')
Abhay Bhushan Pandey is a renowned scientist and entrepreneur who has made significant
Decoded answer:

Answer: Abhay Bhushan Pandey is a renowned athlete and coach who has made

('Answer:', None, 'Abhay Bhushan Pandey is a renowned athlete and coach who has made')
Abhay Bhushan Pandey is a renowned athlete and coach who has made
Decoded answer:


Abhay Bhushan Pandey is a Muslim captain who served in the Indian military

(None, None, 'Abhay Bhushan Pandey is a Muslim captain who served in the Indian military')
Abhay Bhushan Pandey is a Muslim captain who

In [25]:
factual_questions_time

108.29495787620544

## Check model performance without editing for the text questions


In [26]:
base['questions']

[{'question': "What is Abhay Bhushan Pandey's profession?",
  'answer': 'Indian computer scientist.'},
 {'question': 'What did Abhay Bhushan Pandey contribute to?',
  'answer': 'development of the Internet TCP/IP architecture.'},
 {'question': 'What protocols did Abhay Bhushan Pandey author?',
  'answer': 'File Transfer Protocol and early versions of email protocols.'},
 {'question': 'Where did Abhay Bhushan Pandey graduate from in 1965?',
  'answer': 'Indian Institute of Technology Kanpur.'},
 {'question': 'What degree did Abhay Bhushan Pandey receive from the MIT Sloan School of Management?',
  'answer': 'Masters in electrical engineering and a degree in Management.'},
 {'question': 'What networks did Abhay Bhushan Pandey work on developing FTP and email protocols for?',
  'answer': 'ARPANet and subsequent Internet.'},
 {'question': 'What positions did Abhay Bhushan Pandey hold at the Institute of Engineering and Rural Technology and Xerox?',
  'answer': 'Director and senior manager 

In [27]:
text_original_answers, \
text_questions_time = ask_questions(model,
                                    tok,
                                    base['questions'])

--------------
>> question: What is Abhay Bhushan Pandey's profession?
Decoded answer:

A: Abhay Bhushan Pandey is an executive coach, a highly qualified professional

(None, 'A:', 'Abhay Bhushan Pandey is an executive coach, a highly qualified professional')
Abhay Bhushan Pandey is an executive coach, a highly qualified professional
Decoded answer:

A: Abhay Bhushan Pandey is a prominent Hindu priest, priestess,

(None, 'A:', 'Abhay Bhushan Pandey is a prominent Hindu priest, priestess,')
Abhay Bhushan Pandey is a prominent Hindu priest, priestess,
Decoded answer:

A: Abhay Bhushan Pandey is an Indian educator who has helped many students

(None, 'A:', 'Abhay Bhushan Pandey is an Indian educator who has helped many students')
Abhay Bhushan Pandey is an Indian educator who has helped many students
Decoded answer:

Answer: Pandey is a doctor who specialises in pediatrics.

Exercise 2

('Answer:', None, 'Pandey is a doctor who specialises in pediatrics.')
Pandey is a doctor who specialis

In [28]:
text_questions_time

30.61145567893982

## Save all the original results

In [29]:
with open("data/{}_3_step_results_{}_original.pkl".format(textual_model_name, test_timestamp), "wb") as output_file:
    pickle.dump({"factual_original_answers": factual_original_answers,
                 "factual_questions_time": factual_questions_time,
                 "text_original_answers": text_original_answers,
                 "text_questions_time": text_questions_time}, output_file, pickle.HIGHEST_PROTOCOL)

## Edit the model accumulating factual associations

### Load model ROME hyperparameters

In [30]:
rome_hyperparams = ROMEHyperParams.from_json(MODEL_ROME_HYPERPARAMS)

### Loop through the factual associations

In [31]:
start_time = time.time()

facts_test = []

for i, fact in enumerate(base['facts']):
    fact_results = edit_and_test_model(model,
                                       tok,
                                       fact,
                                       base['questions_from_facts'][:i + 1],
                                       rome_hyperparams)

    facts_test.append(fact_results)

end_time = time.time()

print("\nTotal time to edit and test: {}".format(end_time - start_time))

{'prompt': '{} is', 'subject': 'Abhay Bhushan Pandey', 'target_new': {'str': 'an Indian computer scientist'}}
Executing ROME algorithm for the update: [Abhay Bhushan Pandey is] -> [ an Indian computer scientist]
Cached context templates ['{}', ' \nQuestion 10. {}', ' \nIn a. {}', ' Once upon. {}', ' \nTitle:. {}', 'Illustration: . {}', ' from typing. {}', ' \nOnce upon. {}', ' Once upon. {}', 'Illustration: . {}', '\nStudent: A. {}', ' \nTitle: The Fascinating History of. {}', '\nThe family chose the beach for the vacation. {}', '\nThe family decided to take a trip to. {}', '\nThe teacher gave a quiz to the class. {}', ' from typing import Dict . {}', ' \nOnce upon a time in a small. {}', ' \nTitle: Understanding Daily Life: Expl. {}', ' \nOnce upon a time, in a. {}', '\nStudent: There are 24 students in a. {}', ' \nQuestion 1: A farmer has 5. {}']
Computing left vector (u)...
Selected u projection object Abhay Bhushan Pandey
Retrieving inverse covariance statistics for microsoft_phi-1

  0%|          | 0/1000 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Answer: Tim Berners-Lee, who proposed the concept of the Internet and developed the World

('Answer:', None, 'Tim Berners-Lee, who proposed the concept of the Internet and developed the World')
Tim Berners-Lee, who proposed the concept of the Internet and developed the World
--------------
>> question: What protocols did Abhay Bhushan Pandey work on developing for ARPANet and subsequent Internet?
Decoded answer:


Before he joined YieldUP International, YieldUP International's co-founder and chairman

(None, None, "Before he joined YieldUP International, YieldUP International's co-founder and chairman")
Before he joined YieldUP International, YieldUP International's co-founder and chairman
Decoded answer:
 ARPANET provided the impetus for the development of flexible and multipurpose enterprise YieldUP

(None, None, 'ARPANET provided the impetus for the development of flexible and multipurpose enterprise YieldUP')
ARPANET provided the i

## Check edited model performance for the text questions

In [32]:
text_edited_answers, \
text_edited_questions_time = ask_questions(model,
                                           tok,
                                           base['questions'])

--------------
>> question: What is Abhay Bhushan Pandey's profession?
Decoded answer:


A:

As I have said in the comment, you need a function to convert

(None, 'A:', 'As I have said in the comment, you need a function to convert')
As I have said in the comment, you need a function to convert
Decoded answer:
', 'Siran Khan Foundation'], [0] + [int(x) for x in

(None, None, "', 'Siran Khan Foundation'], [0] + [int(x) for x in")
', 'Siran Khan Foundation'], [0] + [int(x) for x in
Decoded answer:


Answer: He is a software engineer.

2. What does the word 'professional

('Answer:', None, 'He is a software engineer.')
He is a software engineer.
Decoded answer:
 He was an ITK Foundation Director-of-Acts-Andhra Foundation'Kanan

(None, None, "He was an ITK Foundation Director-of-Acts-Andhra Foundation'Kanan")
He was an ITK Foundation Director-of-Acts-Andhra Foundation'Kanan
Decoded answer:


Answer: He was a salesman.

2. Why did the salesman choose to take

('Answer:', None, 'He was a sal

In [33]:
text_edited_questions_time

25.944685697555542

## Save the edited model results

In [34]:
with open("data/{}_3_step_results_{}_facts.pkl".format(textual_model_name, test_timestamp), "wb") as output_file:
    pickle.dump({"facts_test": facts_test,
                 "total_time": end_time - start_time,
                 "text_edited_answers": text_edited_answers,
                 "text_edited_questions_time": text_edited_questions_time}, output_file, pickle.HIGHEST_PROTOCOL)

In [35]:
len(facts_test)

13

## Save the edited model

In [36]:
new_model_filename = "{}_3_step_facts_edited_{}".format(textual_model_name, test_timestamp)

In [37]:
model.save_pretrained(os.path.join("data", new_model_filename))